# https://www.kaggle.com/datasets/oyekanmiolamilekan/fake-employee-dataset

In [163]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from datetime import datetime
import hashlib
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

In [164]:
df = pd.read_csv("fake_employee.csv")
df.head()

,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,education_level,certifications,skills,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,Master's Degree,team,"senior, kitchen, president, call, station",3,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,PhD,"whether, some, measure","perhaps, knowledge, officer",4,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,PhD,"job, week","tend, official, red",4,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,PhD,"position, similar","question, their, health, single, whole",2,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,Bachelor's Degree,"table, seem","how, government, finally, east, strong",4,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331


In [165]:
df.info()
df.describe()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   employee_id          1000 non-null   int64 
 1   name                 1000 non-null   object
 2   gender               1000 non-null   object
 3   dates_of_birth       1000 non-null   object
 4   email                1000 non-null   object
 5   phone_number         1000 non-null   object
 6   address              1000 non-null   object
 7   department           1000 non-null   object
 8   job_titles           1000 non-null   object
 9   manager_id           1000 non-null   int64 
 10  hire_date            1000 non-null   object
 11  salary               1000 non-null   int64 
 12  employment_status    1000 non-null   object
 13  employee_type        1000 non-null   object
 14  education_level      1000 non-null   object
 15  certifications       1000 non-null   object
 16  skills 

,employee_id,manager_id,salary,performance_ratings
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,5527.310000,5.454000,81531.299000,2.991000
std,2546.946895,2.837241,23216.141548,1.406021
min,1022.000000,1.000000,40027.000000,1.000000
25%,3351.500000,3.000000,61903.750000,2.000000
50%,5559.000000,5.000000,82902.000000,3.000000
75%,7732.750000,8.000000,101614.750000,4.000000
max,9995.000000,10.000000,119987.000000,5.000000


# Functions

In [166]:

def preprocess_for_ml(df_in):
    df = df_in.copy()
    df = df.drop(columns=['salary'])
    y = df['rounded_salary']
    X = df.drop(columns=['rounded_salary'])  # Drop the target column
    
    # Encode categorical variables
    for col in X.select_dtypes(include='object').columns:
        X[col] = LabelEncoder().fit_transform(X[col])
        
    return X, y


In [167]:

# Function to train and evaluate models
def train_and_evaluate(model, X_train, X_test, y_train, y_test, label=''):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Model ({label}) - MAE: {mae:.2f}")
    return mae


In [168]:

# Training and evaluation function for all models
def evaluate_models(X_train, X_test, y_train, y_test):
    models = [
        (RandomForestRegressor(random_state=42), "Random Forest"),
        (LinearRegression(), "Linear Regression"),
        (DecisionTreeRegressor(random_state=42), "Decision Tree"),
        (GradientBoostingRegressor(random_state=42), "Gradient Boosting")
    ]
    
    results = {}
    for model, label in models:
        mae = train_and_evaluate(model, X_train, X_test, y_train, y_test, label)
        results[label] = mae

    return results


In [169]:
# Use the original dataset (with PII)
df_original = df.copy()



In [170]:
df_original

,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,education_level,certifications,skills,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,Master's Degree,team,"senior, kitchen, president, call, station",3,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,PhD,"whether, some, measure","perhaps, knowledge, officer",4,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,PhD,"job, week","tend, official, red",4,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,PhD,"position, similar","question, their, health, single, whole",2,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,Bachelor's Degree,"table, seem","how, government, finally, east, strong",4,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,Sandra Carpenter,Female,2005-06-10,william72@example.org,591.823.6268,85384 Sullivan Manors Apt. 877\r\nNorth Scottb...,IT,Hydrogeologist,5,...,PhD,something,"week, with, character, source",4,Interesting similar child. Here piece management.,surface,Nicholasburgh,12-8,Active,Joseph Willis: 001-520-862-0025x48408
996,3027,Jackie Church,Male,1971-11-30,morenowyatt@example.net,(640)334-0503,71547 Heather Summit Suite 814\r\nPort Melinda...,HR,"Accountant, chartered public finance",10,...,PhD,"study, election, game","level, price, measure, nor",2,History few news nature agent need human. Vote...,"floor, history",East Michael,12-8,Active,Dawn Wright: 001-650-318-1070
997,3573,Maria Watson,Non-Binary,1979-12-19,chale@example.org,(398)901-9136,"1218 Gonzales Loop\r\nNorth Christineborough, ...",HR,Pension scheme manager,2,...,Associate Degree,"nice, reason, plan","relate, like, production",5,Future benefit quite someone. Half among under...,usually,Heathertown,12-8,Active,Trevor Spencer: 882.361.6741
998,2062,Damon Taylor,Female,2001-08-08,sharris@example.net,451.687.5692x867,"002 Ward Locks\r\nNorth Kimberly, VI 98935",Marketing,"Editor, magazine features",9,...,High School,"receive, product, create","budget, we, high, industry",4,Consider prevent knowledge unit beautiful hold...,"media, shake, treat",West Beverly,12-8,Active,Marilyn Arias: 902-310-6452x4033


In [178]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a countplot for the rounded salary (assuming 'rounded_salary_5000' is the column)
sns.countplot(data=df, x='salary')
plt.title('Distribution of Rounded Salary (5,000)')
plt.xlabel('Rounded Salary')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

ModuleNotFoundError: No module named 'seaborn'

In [171]:

# X, y = preprocess_for_ml(df_original)  # This step should be from your previous code

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Evaluate models
# model_results = evaluate_models(X_train, X_test, y_train, y_test)

# # Print the comparison of MAE results
# print("\nModel Comparison - MAE:")
# for model_name, mae in model_results.items():
#     print(f"{model_name}: {mae:.2f}")


In [172]:
# Round salary to the nearest 5,000
df['rounded_salary'] = (df['salary'] // 10000) * 10000


In [173]:
df['salary']

0       77367
1       81869
2       97907
3      110034
4       70091
        ...  
995     85160
996    109359
997     70527
998     45389
999     64225
Name: salary, Length: 1000, dtype: int64

In [174]:
df['rounded_salary']

0       70000
1       80000
2       90000
3      110000
4       70000
        ...  
995     80000
996    100000
997     70000
998     40000
999     60000
Name: rounded_salary, Length: 1000, dtype: int64

In [176]:
df

,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,certifications,skills,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts,rounded_salary
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,team,"senior, kitchen, president, call, station",3,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176,70000
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,"whether, some, measure","perhaps, knowledge, officer",4,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900,80000
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,"job, week","tend, official, red",4,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971,90000
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,"position, similar","question, their, health, single, whole",2,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439,110000
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,"table, seem","how, government, finally, east, strong",4,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331,70000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,Sandra Carpenter,Female,2005-06-10,william72@example.org,591.823.6268,85384 Sullivan Manors Apt. 877\r\nNorth Scottb...,IT,Hydrogeologist,5,...,something,"week, with, character, source",4,Interesting similar child. Here piece management.,surface,Nicholasburgh,12-8,Active,Joseph Willis: 001-520-862-0025x48408,80000
996,3027,Jackie Church,Male,1971-11-30,morenowyatt@example.net,(640)334-0503,71547 Heather Summit Suite 814\r\nPort Melinda...,HR,"Accountant, chartered public finance",10,...,"study, election, game","level, price, measure, nor",2,History few news nature agent need human. Vote...,"floor, history",East Michael,12-8,Active,Dawn Wright: 001-650-318-1070,100000
997,3573,Maria Watson,Non-Binary,1979-12-19,chale@example.org,(398)901-9136,"1218 Gonzales Loop\r\nNorth Christineborough, ...",HR,Pension scheme manager,2,...,"nice, reason, plan","relate, like, production",5,Future benefit quite someone. Half among under...,usually,Heathertown,12-8,Active,Trevor Spencer: 882.361.6741,70000
998,2062,Damon Taylor,Female,2001-08-08,sharris@example.net,451.687.5692x867,"002 Ward Locks\r\nNorth Kimberly, VI 98935",Marketing,"Editor, magazine features",9,...,"receive, product, create","budget, we, high, industry",4,Consider prevent knowledge unit beautiful hold...,"media, shake, treat",West Beverly,12-8,Active,Marilyn Arias: 902-310-6452x4033,40000


In [177]:
X, y = preprocess_for_ml(df)  # This step should be from your previous code

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Evaluate models
model_results = evaluate_models(X_train, X_test, y_train, y_test)

# Print the comparison of MAE results
print("\nModel Comparison - MAE:")
for model_name, mae in model_results.items():
    print(f"{model_name}: {mae:.2f}")

Model (Random Forest) - MAE: 19387.50
Model (Linear Regression) - MAE: 18993.83
Model (Decision Tree) - MAE: 25250.00
Model (Gradient Boosting) - MAE: 19542.91

Model Comparison - MAE:
Random Forest: 19387.50
Linear Regression: 18993.83
Decision Tree: 25250.00
Gradient Boosting: 19542.91


In [63]:
X

,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,education_level,certifications,skills,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts
0,6612,629,1,278,740,839,493,1,9,10,...,3,813,755,3,681,202,257,1,0,14
1,4793,651,2,728,595,453,701,1,295,8,...,4,910,646,4,642,55,189,1,0,107
2,8832,85,1,599,560,311,582,1,193,6,...,4,413,860,4,545,780,503,1,0,209
3,7044,337,2,881,274,437,353,0,316,5,...,4,594,694,2,991,601,82,0,1,587
4,4498,482,1,844,701,465,48,0,298,2,...,1,806,416,4,26,322,821,2,2,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,831,0,962,943,761,750,3,240,5,...,4,755,945,4,403,810,553,0,0,504
996,3027,358,1,311,651,114,642,2,4,10,...,4,787,504,2,370,303,181,0,0,255
997,3573,613,2,475,132,48,125,2,337,2,...,0,524,712,5,321,875,250,0,0,963
998,2062,223,0,889,823,673,5,4,142,9,...,2,646,131,4,162,462,898,0,0,630


In [ ]:
# # Split into train and test sets

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train a Random Forest model (or any other model)

# model = RandomForestRegressor(random_state=42)
# model.fit(X_train, y_train)

# # Predict and evaluate the model

# y_pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, y_pred)
# print(f"Original Data MAE: {mae:.2f}")


Original Data MAE: 19321.33


# Masking

In [75]:
df_mask = df.copy()  # Preserve full dataset

# Masking phone numbers
df_mask['phone_number_masked'] = df['phone_number'].apply(lambda x: 'XXX-XXX-' + str(x)[-4:])

# Masking email addresses
df_mask['email_masked'] = df['email'].apply(lambda x: 'hidden_' + x.split('@')[-1])

df_mask

,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts,rounded_salary,rounded_salary_5000,phone_number_masked,email_masked
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176,70000,77000,XXX-XXX-6180,hidden_example.net
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900,80000,81000,XXX-XXX-0484,hidden_example.com
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971,90000,97000,XXX-XXX-x248,hidden_example.net
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439,110000,110000,XXX-XXX-5513,hidden_example.net
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331,70000,70000,XXX-XXX-4174,hidden_example.net
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,Sandra Carpenter,Female,2005-06-10,william72@example.org,591.823.6268,85384 Sullivan Manors Apt. 877\r\nNorth Scottb...,IT,Hydrogeologist,5,...,Interesting similar child. Here piece management.,surface,Nicholasburgh,12-8,Active,Joseph Willis: 001-520-862-0025x48408,80000,85000,XXX-XXX-6268,hidden_example.org
996,3027,Jackie Church,Male,1971-11-30,morenowyatt@example.net,(640)334-0503,71547 Heather Summit Suite 814\r\nPort Melinda...,HR,"Accountant, chartered public finance",10,...,History few news nature agent need human. Vote...,"floor, history",East Michael,12-8,Active,Dawn Wright: 001-650-318-1070,100000,109000,XXX-XXX-0503,hidden_example.net
997,3573,Maria Watson,Non-Binary,1979-12-19,chale@example.org,(398)901-9136,"1218 Gonzales Loop\r\nNorth Christineborough, ...",HR,Pension scheme manager,2,...,Future benefit quite someone. Half among under...,usually,Heathertown,12-8,Active,Trevor Spencer: 882.361.6741,70000,70000,XXX-XXX-9136,hidden_example.org
998,2062,Damon Taylor,Female,2001-08-08,sharris@example.net,451.687.5692x867,"002 Ward Locks\r\nNorth Kimberly, VI 98935",Marketing,"Editor, magazine features",9,...,Consider prevent knowledge unit beautiful hold...,"media, shake, treat",West Beverly,12-8,Active,Marilyn Arias: 902-310-6452x4033,40000,45000,XXX-XXX-x867,hidden_example.net


# Generalization

In [76]:
df_gen = df.copy()

# Convert to datetime
df_gen['date_of_birth'] = pd.to_datetime(df['dates_of_birth'], errors='coerce')

# Calculate age
df_gen['age'] = df_gen['date_of_birth'].apply(lambda x: datetime.now().year - x.year if pd.notnull(x) else None)

# Define safe bins that include ALL ages
bin_edges = [0, 20, 30, 40, 50, 60, 70, 120]
age_labels = ['<20', '20s', '30s', '40s', '50s', '60s', '70+']

# Cut into bins safely
df_gen['age_group'] = pd.cut(df_gen['age'], bins=bin_edges, labels=age_labels, right=False)

# df_gen.head()
df_gen[['date_of_birth', 'age', 'age_group']]
# df_gen.isna().sum

,date_of_birth,age,age_group
0,1970-11-14,55,50s
1,1992-08-20,33,30s
2,1986-04-17,39,30s
3,2001-03-15,24,20s
4,1998-10-09,27,20s
...,...,...,...
995,2005-06-10,20,20s
996,1971-11-30,54,50s
997,1979-12-19,46,40s
998,2001-08-08,24,20s


In [77]:
df_gen.isna().sum()

employee_id            0
name                   0
gender                 0
dates_of_birth         0
email                  0
phone_number           0
address                0
department             0
job_titles             0
manager_id             0
hire_date              0
salary                 0
employment_status      0
employee_type          0
education_level        0
certifications         0
skills                 0
performance_ratings    0
work_experience        0
benefits_enrollment    0
city                   0
work_hours             0
employee_status        0
emergency_contacts     0
rounded_salary         0
rounded_salary_5000    0
date_of_birth          0
age                    0
age_group              0
dtype: int64

# Hashing

In [78]:
df_hash = df.copy()  # Preserve full dataset

df_hash['name_hash'] = df['name'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
df_hash['employee_id_hash'] = df['employee_id'].apply(lambda x: hashlib.md5(str(x).encode()).hexdigest())
df_hash

,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts,rounded_salary,rounded_salary_5000,name_hash,employee_id_hash
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176,70000,77000,6a1b015a47088917ec497ef51b5016d924e17734798e38...,d37eb50d868361ea729bb4147eb3c1d8
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900,80000,81000,9ba09996bc6c867c9547e28a65bf9fc7c1df5949523dd4...,b28d8e3ccb1ff9d46fa2d9e70222c9e7
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971,90000,97000,47db4d0a589ed0f170885ce128009e49200131eec4ddd4...,060fd70a06ead2e1079d27612b84aff4
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439,110000,110000,78bf4d5ba2e53e63c569c42859765b78dfb6f37feacd9a...,782086acbe9f48126642e093bf6ba151
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331,70000,70000,f4cae8542f2d9c20e7efbf54d3a76c2067ff0982e19858...,6f3a770e5af1fd4cadc5f004b81e1040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,Sandra Carpenter,Female,2005-06-10,william72@example.org,591.823.6268,85384 Sullivan Manors Apt. 877\r\nNorth Scottb...,IT,Hydrogeologist,5,...,Interesting similar child. Here piece management.,surface,Nicholasburgh,12-8,Active,Joseph Willis: 001-520-862-0025x48408,80000,85000,db06033bfbfb59a509f8cf16f44079f693b6dc43dc82a5...,caaeb10544b465034f389991efc90877
996,3027,Jackie Church,Male,1971-11-30,morenowyatt@example.net,(640)334-0503,71547 Heather Summit Suite 814\r\nPort Melinda...,HR,"Accountant, chartered public finance",10,...,History few news nature agent need human. Vote...,"floor, history",East Michael,12-8,Active,Dawn Wright: 001-650-318-1070,100000,109000,c99aa8c15fc0f3f9dd354a08f318dd879d705ac1e40bb5...,b922ede9c9eb9eabec1c1fecbdecb45d
997,3573,Maria Watson,Non-Binary,1979-12-19,chale@example.org,(398)901-9136,"1218 Gonzales Loop\r\nNorth Christineborough, ...",HR,Pension scheme manager,2,...,Future benefit quite someone. Half among under...,usually,Heathertown,12-8,Active,Trevor Spencer: 882.361.6741,70000,70000,229bd737a2c7ece0fdbc8340500ec5b0a6490871dc87af...,8baca01b732cf56f7ce83df216514363
998,2062,Damon Taylor,Female,2001-08-08,sharris@example.net,451.687.5692x867,"002 Ward Locks\r\nNorth Kimberly, VI 98935",Marketing,"Editor, magazine features",9,...,Consider prevent knowledge unit beautiful hold...,"media, shake, treat",West Beverly,12-8,Active,Marilyn Arias: 902-310-6452x4033,40000,45000,1a73ad681f37f0165e6d980e3c87de32c38260b3918b28...,2d405b367158e3f12d7c1e31a96b3af3


# Pseudonymization 

In [79]:
df_pseudo = df.copy()  # Preserve full dataset

# Create a mapping for pseudonyms
unique_managers = df['manager_id'].unique()
pseudo_ids = ['MGR' + str(i).zfill(3) for i in range(len(unique_managers))]
manager_map = dict(zip(unique_managers, pseudo_ids))

df_pseudo['manager_pseudo'] = df['manager_id'].map(manager_map)
df_pseudo


,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts,rounded_salary,rounded_salary_5000,manager_pseudo
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,3,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176,70000,77000,MGR000
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,4,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900,80000,81000,MGR001
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,4,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971,90000,97000,MGR002
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,2,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439,110000,110000,MGR003
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,4,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331,70000,70000,MGR004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,Sandra Carpenter,Female,2005-06-10,william72@example.org,591.823.6268,85384 Sullivan Manors Apt. 877\r\nNorth Scottb...,IT,Hydrogeologist,5,...,4,Interesting similar child. Here piece management.,surface,Nicholasburgh,12-8,Active,Joseph Willis: 001-520-862-0025x48408,80000,85000,MGR003
996,3027,Jackie Church,Male,1971-11-30,morenowyatt@example.net,(640)334-0503,71547 Heather Summit Suite 814\r\nPort Melinda...,HR,"Accountant, chartered public finance",10,...,2,History few news nature agent need human. Vote...,"floor, history",East Michael,12-8,Active,Dawn Wright: 001-650-318-1070,100000,109000,MGR000
997,3573,Maria Watson,Non-Binary,1979-12-19,chale@example.org,(398)901-9136,"1218 Gonzales Loop\r\nNorth Christineborough, ...",HR,Pension scheme manager,2,...,5,Future benefit quite someone. Half among under...,usually,Heathertown,12-8,Active,Trevor Spencer: 882.361.6741,70000,70000,MGR004
998,2062,Damon Taylor,Female,2001-08-08,sharris@example.net,451.687.5692x867,"002 Ward Locks\r\nNorth Kimberly, VI 98935",Marketing,"Editor, magazine features",9,...,4,Consider prevent knowledge unit beautiful hold...,"media, shake, treat",West Beverly,12-8,Active,Marilyn Arias: 902-310-6452x4033,40000,45000,MGR009


# Randomization 

In [80]:
df_rand = df.copy()  # Preserve full dataset

# Add Gaussian noise
np.random.seed(42)
df_rand['performance_ratings_noisy'] = df['performance_ratings'] + np.random.normal(0, 0.5, df.shape[0])
df_rand['performance_ratings_noisy'] = df_rand['performance_ratings_noisy'].clip(1, 5)
df_rand


,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts,rounded_salary,rounded_salary_5000,performance_ratings_noisy
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,3,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176,70000,77000,3.248357
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,4,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900,80000,81000,3.930868
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,4,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971,90000,97000,4.323844
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,2,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439,110000,110000,2.761515
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,4,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331,70000,70000,3.882923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,Sandra Carpenter,Female,2005-06-10,william72@example.org,591.823.6268,85384 Sullivan Manors Apt. 877\r\nNorth Scottb...,IT,Hydrogeologist,5,...,4,Interesting similar child. Here piece management.,surface,Nicholasburgh,12-8,Active,Joseph Willis: 001-520-862-0025x48408,80000,85000,3.859450
996,3027,Jackie Church,Male,1971-11-30,morenowyatt@example.net,(640)334-0503,71547 Heather Summit Suite 814\r\nPort Melinda...,HR,"Accountant, chartered public finance",10,...,2,History few news nature agent need human. Vote...,"floor, history",East Michael,12-8,Active,Dawn Wright: 001-650-318-1070,100000,109000,2.898843
997,3573,Maria Watson,Non-Binary,1979-12-19,chale@example.org,(398)901-9136,"1218 Gonzales Loop\r\nNorth Christineborough, ...",HR,Pension scheme manager,2,...,5,Future benefit quite someone. Half among under...,usually,Heathertown,12-8,Active,Trevor Spencer: 882.361.6741,70000,70000,5.000000
998,2062,Damon Taylor,Female,2001-08-08,sharris@example.net,451.687.5692x867,"002 Ward Locks\r\nNorth Kimberly, VI 98935",Marketing,"Editor, magazine features",9,...,4,Consider prevent knowledge unit beautiful hold...,"media, shake, treat",West Beverly,12-8,Active,Marilyn Arias: 902-310-6452x4033,40000,45000,3.714411


In [82]:
# Dataset 1: Original (minimal cleaning, drop only name/email if you want)
df_original = df.copy()
# df_original.drop(columns=['name', 'email'], inplace=True)  # optional

# Dataset 2: PII anonymized
df_pii = df.copy()
# Add PII anonymized columns
df_pii['phone_number_masked'] = df_mask['phone_number_masked']
df_pii['email_masked'] = df_mask['email_masked']
df_pii['name_hash'] = df_hash['name_hash']
df_pii['employee_id_hash'] = df_hash['employee_id_hash']
df_pii['age_group'] = df_gen['age_group']
df_pii.drop(columns=['name', 'email', 'phone_number', 'employee_id', 'dates_of_birth'], inplace=True)

def keep_alpha_prefix(postal):
    if not isinstance(postal, str):
        return 'Unknown'
    match = re.match(r'^[A-Za-z]+', postal)
    return match.group(0) if match else 'Unknown'

df_pii['postal_code'] = df_pii['address'].apply(lambda x: x.split(',')[-1].strip() if isinstance(x, str) else 'Unknown')
df_pii['postal_code_prefix'] = df_pii['postal_code'].apply(keep_alpha_prefix)
df_pii['address'] = df_pii['postal_code_prefix']

# Hash emergency contact names
df_pii['emergency_contacts'] = df_pii['emergency_contacts'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

df_pii.drop(columns=['postal_code', 'postal_code_prefix'], inplace=True)


# Dataset 3: PII + QID anonymized
df_pii_qid = df_pii.copy()
df_pii_qid['manager_pseudo'] = df_pseudo['manager_pseudo']
df_pii_qid['performance_ratings_noisy'] = df_rand['performance_ratings_noisy']
# Add any QID generalization like job_titles and city here:
# e.g., df_pii_qid['job_title_group'] = your_generalization_function(df['job_titles'])
# Drop original columns you generalized/anonymized:
df_pii_qid.drop(columns=['manager_id', 'performance_ratings', 'job_titles', 'city'], inplace=True)


In [83]:
# Mapping age groups to their corresponding start of the range (numeric value)
age_group_mapping = {
    '20s': 20,
    '30s': 30,
    '40s': 40,
    '50s': 50,
    '60s': 60,
    '70+': 70  # or set it to a higher value like 75 or 80, based on dataset
}

# Apply the mapping to your dataset
df_pii['age_group'] = df_pii['age_group'].map(age_group_mapping)

In [84]:
df_pii.head()
df_pii.to_csv("pii.csv", index=False)

In [85]:
# Check for NaN or missing values
df_pii.isna().sum()


gender                 0
address                0
department             0
job_titles             0
manager_id             0
hire_date              0
salary                 0
employment_status      0
employee_type          0
education_level        0
certifications         0
skills                 0
performance_ratings    0
work_experience        0
benefits_enrollment    0
city                   0
work_hours             0
employee_status        0
emergency_contacts     0
rounded_salary         0
rounded_salary_5000    0
phone_number_masked    0
email_masked           0
name_hash              0
employee_id_hash       0
age_group              0
dtype: int64

In [ ]:

# def preprocess_for_ml(df_in):
#     df = df_in.copy()
#     y = df['salary']
#     X = df.drop(columns=['salary'])  # Drop the target column
    
#     # Encode categorical variables
#     for col in X.select_dtypes(include='object').columns:
#         X[col] = LabelEncoder().fit_transform(X[col])
        
#     return X, y


In [86]:
# Train/test split if not done yet

X, y = preprocess_for_ml(df)  # This step should be from your previous code

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Evaluate models
model_results = evaluate_models(X_train, X_test, y_train, y_test)

# Print the comparison of MAE results
print("\nModel Comparison - MAE:")
for model_name, mae in model_results.items():
    print(f"{model_name}: {mae:.2f}")

Model (Random Forest) - MAE: 287.00
Model (Linear Regression) - MAE: 258.81
Model (Decision Tree) - MAE: 360.88
Model (Gradient Boosting) - MAE: 277.13

Model Comparison - MAE:
Random Forest: 287.00
Linear Regression: 258.81
Decision Tree: 360.88
Gradient Boosting: 277.13


In [91]:
df

,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,skills,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts,rounded_salary,rounded_salary_5000
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,"senior, kitchen, president, call, station",3,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176,70000,77000
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,"perhaps, knowledge, officer",4,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900,80000,81000
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,"tend, official, red",4,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971,90000,97000
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,"question, their, health, single, whole",2,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439,110000,110000
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,"how, government, finally, east, strong",4,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331,70000,70000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,Sandra Carpenter,Female,2005-06-10,william72@example.org,591.823.6268,85384 Sullivan Manors Apt. 877\r\nNorth Scottb...,IT,Hydrogeologist,5,...,"week, with, character, source",4,Interesting similar child. Here piece management.,surface,Nicholasburgh,12-8,Active,Joseph Willis: 001-520-862-0025x48408,80000,85000
996,3027,Jackie Church,Male,1971-11-30,morenowyatt@example.net,(640)334-0503,71547 Heather Summit Suite 814\r\nPort Melinda...,HR,"Accountant, chartered public finance",10,...,"level, price, measure, nor",2,History few news nature agent need human. Vote...,"floor, history",East Michael,12-8,Active,Dawn Wright: 001-650-318-1070,100000,109000
997,3573,Maria Watson,Non-Binary,1979-12-19,chale@example.org,(398)901-9136,"1218 Gonzales Loop\r\nNorth Christineborough, ...",HR,Pension scheme manager,2,...,"relate, like, production",5,Future benefit quite someone. Half among under...,usually,Heathertown,12-8,Active,Trevor Spencer: 882.361.6741,70000,70000
998,2062,Damon Taylor,Female,2001-08-08,sharris@example.net,451.687.5692x867,"002 Ward Locks\r\nNorth Kimberly, VI 98935",Marketing,"Editor, magazine features",9,...,"budget, we, high, industry",4,Consider prevent knowledge unit beautiful hold...,"media, shake, treat",West Beverly,12-8,Active,Marilyn Arias: 902-310-6452x4033,40000,45000


In [90]:
df["salary", "rounded_salary"]

KeyError: ('salary', 'rounded_salary')

In [88]:
df_pii

,gender,address,department,job_titles,manager_id,hire_date,salary,employment_status,employee_type,education_level,...,work_hours,employee_status,emergency_contacts,rounded_salary,rounded_salary_5000,phone_number_masked,email_masked,name_hash,employee_id_hash,age_group
0,Male,PW,Finance,Administrator,10,2023-04-06,77367,Part-time,Regular,Master's Degree,...,9-5,Active,1a8880eeae59b3404b2686ca7d704111a285835ee8184f...,70000,77000,XXX-XXX-6180,hidden_example.net,6a1b015a47088917ec497ef51b5016d924e17734798e38...,d37eb50d868361ea729bb4147eb3c1d8,50.0
1,Non-Binary,DE,Finance,Mental health nurse,8,2022-05-26,81869,Full-time,Temporary,PhD,...,9-5,Active,615099388f299dad3fc32beb42b544d979f3fceb58e7c3...,80000,81000,XXX-XXX-0484,hidden_example.com,9ba09996bc6c867c9547e28a65bf9fc7c1df5949523dd4...,b28d8e3ccb1ff9d46fa2d9e70222c9e7,30.0
2,Male,LA,Finance,Field seismologist,6,2020-06-11,97907,Part-time,Temporary,PhD,...,9-5,Active,c2d93a58a8e5aec9dcd337c60bc98e9ceba8c73e40859f...,90000,97000,XXX-XXX-x248,hidden_example.net,47db4d0a589ed0f170885ce128009e49200131eec4ddd4...,060fd70a06ead2e1079d27612b84aff4,30.0
3,Non-Binary,OR,Engineering,"Nurse, children's",5,2020-11-25,110034,Part-time,Regular,PhD,...,12-8,On Leave,b698cf4a21910aff2ece23a3c3391e368d1cb50379e723...,110000,110000,XXX-XXX-5513,hidden_example.net,78bf4d5ba2e53e63c569c42859765b78dfb6f37feacd9a...,782086acbe9f48126642e093bf6ba151,20.0
4,Male,IN,Engineering,Meteorologist,2,2021-01-12,70091,Contractor,Contract,Bachelor's Degree,...,Night Shift,Terminated,364253bc6b8c32714e40bf566d7c11fb43be228a0d1b26...,70000,70000,XXX-XXX-4174,hidden_example.net,f4cae8542f2d9c20e7efbf54d3a76c2067ff0982e19858...,6f3a770e5af1fd4cadc5f004b81e1040,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Female,LA,IT,Hydrogeologist,5,2022-10-14,85160,Part-time,Regular,PhD,...,12-8,Active,c75b1165b2eb23b4ff011d2eb13e69432c28dba6cb73dc...,80000,85000,XXX-XXX-6268,hidden_example.org,db06033bfbfb59a509f8cf16f44079f693b6dc43dc82a5...,caaeb10544b465034f389991efc90877,20.0
996,Male,NH,HR,"Accountant, chartered public finance",10,2020-10-07,109359,Part-time,Regular,PhD,...,12-8,Active,818415a8ea5ed6c4c9eb7e64140dee47cfeda89739c7a4...,100000,109000,XXX-XXX-0503,hidden_example.net,c99aa8c15fc0f3f9dd354a08f318dd879d705ac1e40bb5...,b922ede9c9eb9eabec1c1fecbdecb45d,50.0
997,Non-Binary,WI,HR,Pension scheme manager,2,2020-09-04,70527,Contractor,Temporary,Associate Degree,...,12-8,Active,873dc4755a11c5f116a9301f62da00e136482657262e62...,70000,70000,XXX-XXX-9136,hidden_example.org,229bd737a2c7ece0fdbc8340500ec5b0a6490871dc87af...,8baca01b732cf56f7ce83df216514363,40.0
998,Female,VI,Marketing,"Editor, magazine features",9,2019-08-03,45389,Full-time,Regular,High School,...,12-8,Active,487f5fd21c3ae79d3c68775e5feb66aaeeee5a9671df02...,40000,45000,XXX-XXX-x867,hidden_example.net,1a73ad681f37f0165e6d980e3c87de32c38260b3918b28...,2d405b367158e3f12d7c1e31a96b3af3,20.0


In [87]:
# Train/test split if not done yet

X, y = preprocess_for_ml(df_pii)  # This step should be from your previous code

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Evaluate models
model_results = evaluate_models(X_train, X_test, y_train, y_test)

# Print the comparison of MAE results
print("\nModel Comparison - MAE:")
for model_name, mae in model_results.items():
    print(f"{model_name}: {mae:.2f}")

Model (Random Forest) - MAE: 268.64
Model (Linear Regression) - MAE: 256.19
Model (Decision Tree) - MAE: 320.12
Model (Gradient Boosting) - MAE: 277.30

Model Comparison - MAE:
Random Forest: 268.64
Linear Regression: 256.19
Decision Tree: 320.12
Gradient Boosting: 277.30


In [ ]:
# Train/test split if not done yet

# X, y = preprocess_for_ml(df_pii)  # This step should be from your previous code

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Evaluate models
# model_results = evaluate_models(X_train, X_test, y_train, y_test)

# # Print the comparison of MAE results
# print("\nModel Comparison - MAE:")
# for model_name, mae in model_results.items():
#     print(f"{model_name}: {mae:.2f}")


Model (Random Forest) - MAE: 19753.15
Model (Linear Regression) - MAE: 19186.81
Model (Decision Tree) - MAE: 27533.17
Model (Gradient Boosting) - MAE: 20276.60

Model Comparison - MAE:
Random Forest: 19753.15
Linear Regression: 19186.81
Decision Tree: 27533.17
Gradient Boosting: 20276.60


In [42]:
df['salary'].describe()


count      1000.000000
mean      81531.299000
std       23216.141548
min       40027.000000
25%       61903.750000
50%       82902.000000
75%      101614.750000
max      119987.000000
Name: salary, dtype: float64

In [21]:
df_pii_qid.head()

,gender,address,department,hire_date,salary,employment_status,employee_type,education_level,certifications,skills,...,work_hours,employee_status,emergency_contacts,phone_number_masked,email_masked,name_hash,employee_id_hash,age_group,manager_pseudo,performance_ratings_noisy
0,Male,PW,Finance,2023-04-06,77367,Part-time,Regular,Master's Degree,team,"senior, kitchen, president, call, station",...,9-5,Active,1a8880eeae59b3404b2686ca7d704111a285835ee8184f...,XXX-XXX-6180,hidden_example.net,6a1b015a47088917ec497ef51b5016d924e17734798e38...,d37eb50d868361ea729bb4147eb3c1d8,50s,MGR000,3.248357
1,Non-Binary,DE,Finance,2022-05-26,81869,Full-time,Temporary,PhD,"whether, some, measure","perhaps, knowledge, officer",...,9-5,Active,615099388f299dad3fc32beb42b544d979f3fceb58e7c3...,XXX-XXX-0484,hidden_example.com,9ba09996bc6c867c9547e28a65bf9fc7c1df5949523dd4...,b28d8e3ccb1ff9d46fa2d9e70222c9e7,30s,MGR001,3.930868
2,Male,LA,Finance,2020-06-11,97907,Part-time,Temporary,PhD,"job, week","tend, official, red",...,9-5,Active,c2d93a58a8e5aec9dcd337c60bc98e9ceba8c73e40859f...,XXX-XXX-x248,hidden_example.net,47db4d0a589ed0f170885ce128009e49200131eec4ddd4...,060fd70a06ead2e1079d27612b84aff4,30s,MGR002,4.323844
3,Non-Binary,OR,Engineering,2020-11-25,110034,Part-time,Regular,PhD,"position, similar","question, their, health, single, whole",...,12-8,On Leave,b698cf4a21910aff2ece23a3c3391e368d1cb50379e723...,XXX-XXX-5513,hidden_example.net,78bf4d5ba2e53e63c569c42859765b78dfb6f37feacd9a...,782086acbe9f48126642e093bf6ba151,20s,MGR003,2.761515
4,Male,IN,Engineering,2021-01-12,70091,Contractor,Contract,Bachelor's Degree,"table, seem","how, government, finally, east, strong",...,Night Shift,Terminated,364253bc6b8c32714e40bf566d7c11fb43be228a0d1b26...,XXX-XXX-4174,hidden_example.net,f4cae8542f2d9c20e7efbf54d3a76c2067ff0982e19858...,6f3a770e5af1fd4cadc5f004b81e1040,20s,MGR004,3.882923


In [44]:
df

,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,education_level,certifications,skills,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,Master's Degree,team,"senior, kitchen, president, call, station",3,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,PhD,"whether, some, measure","perhaps, knowledge, officer",4,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,PhD,"job, week","tend, official, red",4,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,PhD,"position, similar","question, their, health, single, whole",2,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,Bachelor's Degree,"table, seem","how, government, finally, east, strong",4,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,Sandra Carpenter,Female,2005-06-10,william72@example.org,591.823.6268,85384 Sullivan Manors Apt. 877\r\nNorth Scottb...,IT,Hydrogeologist,5,...,PhD,something,"week, with, character, source",4,Interesting similar child. Here piece management.,surface,Nicholasburgh,12-8,Active,Joseph Willis: 001-520-862-0025x48408
996,3027,Jackie Church,Male,1971-11-30,morenowyatt@example.net,(640)334-0503,71547 Heather Summit Suite 814\r\nPort Melinda...,HR,"Accountant, chartered public finance",10,...,PhD,"study, election, game","level, price, measure, nor",2,History few news nature agent need human. Vote...,"floor, history",East Michael,12-8,Active,Dawn Wright: 001-650-318-1070
997,3573,Maria Watson,Non-Binary,1979-12-19,chale@example.org,(398)901-9136,"1218 Gonzales Loop\r\nNorth Christineborough, ...",HR,Pension scheme manager,2,...,Associate Degree,"nice, reason, plan","relate, like, production",5,Future benefit quite someone. Half among under...,usually,Heathertown,12-8,Active,Trevor Spencer: 882.361.6741
998,2062,Damon Taylor,Female,2001-08-08,sharris@example.net,451.687.5692x867,"002 Ward Locks\r\nNorth Kimberly, VI 98935",Marketing,"Editor, magazine features",9,...,High School,"receive, product, create","budget, we, high, industry",4,Consider prevent knowledge unit beautiful hold...,"media, shake, treat",West Beverly,12-8,Active,Marilyn Arias: 902-310-6452x4033
